 # Agregar ruido gausiano con niveles de dB específicos 
 
Esta función suma ruido gaussiano a pistas de audio, calcula su kurtosis, guardar el resultado en un audio .wav y crear una tabla con las kurtosis obtenidas para cada ruido sumando. Los niveles peak de cada audio deben estar enlistados en un excel para ser leidos por la función. Los parámetros de entrada son los siguietnes:
    1. folderPath: Path to folder with audios.
    2. excel_path: Path to the Excel file containing Lpeak values.
    3. newpath: Path to the folder where the modified audio files will be saved.
    4. lowerdB: Minimum dB level for Gaussian noise (default: 80).
    5. upperdB: Maximum dB level for Gaussian noise (default: 120).
    6. d_s: Duration of the noise signal in seconds (default: 1).
    7. s_r: Sampling rate in samples per second (default: 44100).

In [1]:
import os
import numpy as np
import pandas as pd
import soundfile as sf
from scipy.stats import kurtosis

def kurtosis_plus_gauss(folderPath, excel_path, newpath, lowerdB=80, upperdB=120, d_s=1, s_r=44100):
   
    os.makedirs(newpath, exist_ok=True)

    # Read audio files
    audioFiles = [f for f in os.listdir(folderPath) if f.endswith('.wav')]
    L = len(audioFiles)
    list_dB = np.arange(lowerdB, upperdB + 1, 5)
    list_dB_full = ["Original"] + list(list_dB)

    # Initialize storage for audio data and kurtosis values
    audioK = np.zeros((L, len(list_dB) + 1))

    # Read Lpeak values from the Excel file
    LpeakData = pd.read_excel(excel_path, usecols='C', skiprows=2, nrows=22)
    Lpeak = LpeakData.values.flatten()

    p_ref = 20e-6  # Reference sound pressure level in Pascals

    # Process each audio file
    for a, name in enumerate(audioFiles):
        audio, fs = sf.read(os.path.join(folderPath, name))

        if a < len(Lpeak):
            Lpeaki = Lpeak[a]
            p_peak = p_ref * 10 ** (Lpeaki / 20)

            # Scaling
            p = audio * (p_peak / np.max(audio))
            l = len(p)
            t = np.arange(0, l / fs, 1 / fs)

            # Calculate kurtosis
            k = kurtosis(p, fisher=False)
            audioK[a, 0] = k

            for i, dBi in enumerate(list_dB):
                Leq_linear = 2e-5 * 10 ** (dBi / 20)  # Convert dB to linear scale
                sigma = Leq_linear  # Standard deviation of Gaussian distribution
                num_samp = d_s * s_r  # Number of samples
                gauss_noise = sigma * np.random.randn(num_samp)  # Generate Gaussian noise

                # Add signals
                gnI = p + gauss_noise

                ki = kurtosis(gnI, fisher=False)  # Kurtosis of the new signal
                audioK[a, i + 1] = ki

                # Normalize
                maxp = np.max(np.abs(gnI))
                Lpeak_new = 10 * np.log10((maxp ** 2) / (p_ref ** 2))
                gnI = gnI / maxp

                # Save the new audio file
                baseName, _ = os.path.splitext(name)
                filename = f"{baseName}_{dBi}_{Lpeak_new:.2f}.wav"
                sf.write(os.path.join(newpath, filename), gnI, fs)

                list_dB_full[i + 1] = dBi

    # Create table
    Tablek = pd.DataFrame(audioK, index=audioFiles, columns=list_dB_full)

    return Tablek



In [2]:

# Example use
folderPath = '/Users/chiaravald/Desktop/ACUS340/Audios_Drums'
excel_path = '/Users/chiaravald/Desktop/ACUS340/Audios_Drums/Drums.xlsx'
newpath = '/Users/chiaravald/Desktop/ACUS340/Audio_Drums_Impulsos_wGauss'
lowerdB = 80
upperdB = 120
Tablek = kurtosis_plus_gauss(folderPath, excel_path, newpath, lowerdB, upperdB)
print(Tablek)


                           Original         80         85         90  \
11FloorTom16LeftEar.wav   25.635174   3.540455   3.043898   3.014052   
14Ride20RightEar.wav      87.062857  86.787276  86.126876  83.861868   
6Tom10RightEar.wav         3.894662   3.894610   3.894841   3.893285   
7Tom12LeftEar.wav          6.643005   6.624853   6.582282   6.450970   
15Crash18LeftEar.wav       5.851270   5.848862   5.836216   5.799162   
1KickLeftEar.wav          25.680559  25.654291  25.673249  25.294239   
21HiHatOpenLeftEar.wav    14.197763  14.157869  14.154851  14.091314   
10FloorTom14RightEar.wav  11.698074  11.694070  11.686467  11.648615   
3SnareLeftEar.wav         54.813775  54.751191  54.686476  54.479273   
18Crash16RightEar.wav      6.575207   6.574388   6.567543   6.564496   
2KickRightEar.wav         49.666109  49.569772  49.490106  49.139188   
22HiHatOpenRightEar.wav   12.463554  12.393158  12.277298  11.858698   
19HiHatCloseLeftEar.wav   60.935275  59.282103  57.475578  50.76